In [1]:
import sys
import os
import subprocess
import shutil
# use shutil.rmtree(<path_to_folder>) to do an rm -r on the folder

import json
import csv
# import yaml
from pathlib import Path

import pandas as pd
import numpy as np

# import matplotlib as mpl
# from cycler import cycler

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set(style="darkgrid")

# This useful if I want to give unique names to directories or files
import datetime
def curr_timestamp():
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    return formatted_datetime

print('Initial imports successful')

Initial imports successful


In [5]:
# imitating example in 
# https://github.com/JJguri/bestiapop/blob/master/sample-data/ExampleMapsTasmania.ipynb

from bestiapop import bestiapop as bp

working_dir = './'

lonlat_file = working_dir + 'centroids.csv'
lonlat_df = pd.read_csv(lonlat_file)

year_range = '1981-2010' # using this string format rather than, e.g., np.array([2010,2011,2012])

# these are the only climate variables supported by BestiaPop on NASAPOWER data
climate_variables = np.array(['daily_rain', 'max_temp', 'min_temp', 'radiation'])

# on NASAPOWER, it works on round numbers and xx.5 numbers only; 
# if you give it a coordinate such as 145.63 it will round to 145.5 and/or 146

action = 'generate-climate-file'
data_source = 'nasapower'
output_type = 'dataframe'

lat_range = [lonlat_df["lat"].min(), lonlat_df["lat"].max()]
lon_range = [lonlat_df["long"].min(), lonlat_df["long"].max()]

output_path_sector = output_path + "/" + str(sector) + "/";
Path(output_path_sector).mkdir(parents=True, exist_ok=True);

print('Fetching data for {}/{}'.format(lat_range, lon_range))
climate_data = bp.CLIMATEBEAST(
    action=action,
    data_source=data_source,
    output_path=None,
    output_type=output_type,
    input_path=None,
    climate_variables=climate_variables,
    year_range=year_range,
    lat_range=lat_range,
    lon_range=lon_range,
    multiprocessing=None
)

df = climate_data.process_records(action)



2023-05-22 17:59:27 - POPBEAST - Extracting data and converting to dataframe format


Fetching data for [38.79999542236328, 39.38333511352539]/[-122.13333892822264, -121.63333892822264]
Total Progress: 100%|##########| 2/2 [00:27<00:00, 13.57s/it]


In [6]:
df

,lon,lat,year,day,radn,maxt,mint,rain
0,-122.13,38.8,1981,1,0.00,17.81,5.72,0.24
1,-122.13,38.8,1981,2,0.00,13.12,5.53,2.01
2,-122.13,38.8,1981,3,0.00,15.65,7.00,3.38
3,-122.13,38.8,1981,4,0.00,17.73,5.69,0.07
4,-122.13,38.8,1981,5,0.00,17.63,2.34,0.18
...,...,...,...,...,...,...,...,...
43823,-121.63,39.3,2010,361,7.21,12.16,5.15,0.25
43824,-121.63,39.3,2010,362,2.50,8.22,5.32,32.11
43825,-121.63,39.3,2010,363,8.01,8.00,0.48,6.05
43826,-121.63,39.3,2010,364,9.35,8.39,-0.74,0.01


In [15]:
from scipy.interpolate import interp2d

In [90]:
sectors = {}

for sector in lonlat_df.itertuples():
    sectors[sector.sector] = {
        "year": [],
        "day": [],
        "radn": [],
        "maxt": [],
        "mint": [],
        "rain": [],
    }

In [91]:
def interpolate_and_append(rows):
  radn = interp2d(rows["lon"], rows["lat"], rows["radn"])
  maxt = interp2d(rows["lon"], rows["lat"], rows["maxt"])
  mint = interp2d(rows["lon"], rows["lat"], rows["mint"])
  rain = interp2d(rows["lon"], rows["lat"], rows["rain"])
  for row in lonlat_df.itertuples():
    sector = sectors[row.sector];
    sector["year"].append(rows["year"].iloc[1])
    sector["day"].append(rows["day"].iloc[1])
    sector["radn"].append(radn(row.long, row.lat)[0])
    sector["maxt"].append(maxt(row.long, row.lat)[0])
    sector["mint"].append(mint(row.long, row.lat)[0])
    sector["rain"].append(rain(row.long, row.lat)[0])


df.groupby(["year", "day"]).apply(interpolate_and_append)

""


In [92]:
for sector in lonlat_df.itertuples():
    values = sectors[sector.sector];
    f = open(year_range + "-sector-" + str(sector.sector) + ".met", "w")
    f.write("[weather.met.weather]\n")
    f.write("Latitude=")
    f.write(str(sector.lat))
    f.write("\n")
    f.write("Longitude=")
    f.write(str(sector.long))
    f.write("\n")
    f.write("\n")
    f.write("year day radn maxt mint rain\n")
    f.write("() () (MJ^m2) (oC) (oC) (mm)\n")
    for i in range(0, len(values["year"])):
      f.write(str(values["year"][i]))
      f.write(" ")
      f.write(str(values["day"][i]))
      f.write(" ")
      f.write(str(values["radn"][i]))
      f.write(" ")
      f.write(str(values["maxt"][i]))
      f.write(" ")
      f.write(str(values["mint"][i]))
      f.write(" ")
      f.write(str(values["rain"][i]))
      f.write("\n")

    f.close()